In [ ]:
!pip3 install transformers
!pip3 install emoji
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer 
import tensorflow as tf
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import locale

import os
import time

In [ ]:
# Clone github repo containing the data as csv file
! rm -rf covid_data
! git clone https://github.com/PatrickNiccolai2/covid_data

Cloning into 'covid_data'...
remote: Enumerating objects: 2040, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 2040 (delta 24), reused 90 (delta 22), pack-reused 1948
Receiving objects: 100% (2040/2040), 334.22 MiB | 30.70 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [ ]:
# Download the tweets with sentiment data
!unzip /content/covid_data/tweets_with_lang -d /content/covid_data/unzip_tweets_with_lang

Streaming output truncated to the last 5000 lines.
  inflating: /content/covid_data/unzip_tweets_with_lang/tweets_with_lang/lang_sent_shuffled_tweet_80172.txt  
  inflating: /content/covid_data/unzip_tweets_with_lang/tweets_with_lang/lang_sent_shuffled_tweet_8455.txt  
  inflating: /content/covid_data/unzip_tweets_with_lang/tweets_with_lang/lang_sent_shuffled_tweet_54445.txt  
  inflating: /content/covid_data/unzip_tweets_with_lang/tweets_with_lang/lang_sent_shuffled_tweet_65840.txt  
  inflating: /content/covid_data/unzip_tweets_with_lang/tweets_with_lang/lang_sent_shuffled_tweet_7766.txt  
  inflating: /content/covid_data/unzip_tweets_with_lang/tweets_with_lang/lang_sent_shuffled_tweet_74795.txt  
  inflating: /content/covid_data/unzip_tweets_with_lang/tweets_with_lang/lang_sent_shuffled_tweet_23815.txt  
  inflating: /content/covid_data/unzip_tweets_with_lang/tweets_with_lang/lang_sent_shuffled_tweet_43139.txt  
  inflating: /content/covid_data/unzip_tweets_with_lang/tweets_with_lan

In [ ]:
# Read all of the tweets + other data into an array
all_files_text = []

# Iterate through folder we just unzipped
for filename in os.listdir("/content/covid_data/unzip_tweets_with_lang/tweets_with_lang"):
  if(filename[0] == "l"):
    # Open each file and read it
    file_path = "/content/covid_data/unzip_tweets_with_lang/tweets_with_lang/" + filename
    file = open(file_path, "r")
    lines = file.read()

    # Split the file into different parts
    lines_arr = lines.split("\n\n")
    lang = lines_arr[-1]
    sent = lines_arr[-2]
    id = lines_arr[-3]
    place = lines_arr[-5]
    coords = lines_arr[-5]
    time = lines_arr[-6]

    tweet_text = ""
    for i in range(7, len(lines_arr) + 1):
      tweet_text = tweet_text + lines_arr[-i]

    # Search the tweet text for a manufacturer, if one is mentioned
    if("johnson" not in tweet_text.lower()):
      man = "no_man"
      if("sputnik" in tweet_text.lower()):
        man = "sputnik"
      if("sinopharm" in tweet_text.lower() or "sinovac" in tweet_text.lower()):
        man = "sino"
      if("moderna" in tweet_text.lower() or "spikevax" in tweet_text.lower()):
        man = "moderna"
      if("pfizer" in tweet_text.lower() or "biontech" in tweet_text.lower() or "comirnaty" in tweet_text.lower()):
        man = "pfizer"
      if("janssen" in tweet_text.lower()):
        man = "janssen"
      if("astrazeneca" in tweet_text.lower() or "vaxzevria" in tweet_text.lower()):
        man = "astra"
        
      # Add all the info from the file to all_files_text
      full_tweet = [tweet_text, time, coords, place, id, man, sent, lang]
      all_files_text.append(full_tweet)
    file.close()


In [ ]:
# This is how many tweets we have
print(len(all_files_text))

114549


In [ ]:
# Create a dataframe with all of the data
all_tweets_df = pd.DataFrame(all_files_text, columns=["tweet_text", "time", "coords", "place", "tweet_id", "man", "sentiment", "language"])
all_tweets_df = all_tweets_df.drop_duplicates(subset='tweet_id', keep="first")
all_tweets_df = all_tweets_df.loc[all_tweets_df["language"] == "en"]

In [ ]:
print(all_tweets_df.head(10))

                                           tweet_text  \
0   @PtboExaminer People’s mental health is suffer...   
1   #LargestVaccineDriveVaccination to be done on ...   
2   @hmarie52 Gates: snake oil salesman wants us t...   
3   Soon I can go back to Broadway where all cast,...   
4   Instead allocate the PKR 100 million to buy va...   
6   A huge shoutout to all the heroes who particip...   
7   @drharshvardhan @MoHFW_INDIA @PMOIndia Disclos...   
9   @datHockeyHair if the point of getting the vac...   
10  @modifiedresins @UniofOxford @VaccineTrials @O...   
11  In the last 24 hours we’ve gotten word that tw...   

                         time coords place             tweet_id     man  \
0   2021-05-08 15:54:11+00:00   None  None  1391058861289885696  no_man   
1   2021-04-01 09:44:28+00:00   None  None  1377557468587257858  no_man   
2   2020-04-06 17:19:34+00:00   None  None  1247212363704602624  no_man   
3   2021-08-03 00:56:01+00:00   None  None  1422360571182256133  no_man 

In [ ]:
# Convert strings to datetime so they can be used
def str_to_datetime(in_str):
  date = in_str.split(" ")[0]
  out_datetime = datetime.strptime(date,"%Y-%m-%d")
  return out_datetime

all_tweets_df["datetime"] = all_tweets_df["time"].apply(str_to_datetime)

In [ ]:
# The sentiment is an array containing 3 values, convert it to one value
def likely_sent(sent_str):
  sent_str = sent_str[2:-2]
  sent_arr = sent_str.split()
  sent_arr = [float(x) for x in sent_arr]
  return max(range(len(sent_arr)), key=sent_arr.__getitem__)

all_tweets_df["sent_category"] = all_tweets_df["sentiment"].apply(likely_sent)

In [ ]:
print(all_tweets_df.head(10))

                                           tweet_text  \
0   @PtboExaminer People’s mental health is suffer...   
1   #LargestVaccineDriveVaccination to be done on ...   
2   @hmarie52 Gates: snake oil salesman wants us t...   
3   Soon I can go back to Broadway where all cast,...   
4   Instead allocate the PKR 100 million to buy va...   
6   A huge shoutout to all the heroes who particip...   
7   @drharshvardhan @MoHFW_INDIA @PMOIndia Disclos...   
9   @datHockeyHair if the point of getting the vac...   
10  @modifiedresins @UniofOxford @VaccineTrials @O...   
11  In the last 24 hours we’ve gotten word that tw...   

                         time coords place             tweet_id     man  \
0   2021-05-08 15:54:11+00:00   None  None  1391058861289885696  no_man   
1   2021-04-01 09:44:28+00:00   None  None  1377557468587257858  no_man   
2   2020-04-06 17:19:34+00:00   None  None  1247212363704602624  no_man   
3   2021-08-03 00:56:01+00:00   None  None  1422360571182256133  no_man 

In [ ]:
# This cell counts how many tweets mention a manufacturer

num_sputnik = 0
num_sino = 0
num_moderna = 0
num_pfizer = 0
num_janssen = 0
num_astra = 0

sputnik_arr = []
sino_arr = []
moderna_arr = []
pfizer_arr = []
janssen_arr = []
astra_arr = []
for file in all_files_text:
  tweet_text = file[0]
  if("sputnik" in tweet_text.lower()):
    num_sputnik += 1
    sputnik_arr.append(file)
  if("sinopharm" in tweet_text.lower() or "sinovac" in tweet_text.lower()):
    num_sino += 1
    sino_arr.append(file)
  if("moderna" in tweet_text.lower() or "spikevax" in tweet_text.lower()):
    num_moderna += 1
    moderna_arr.append(file)
  if("pfizer" in tweet_text.lower() or "biontech" in tweet_text.lower() or "comirnaty" in tweet_text.lower()):
    num_pfizer += 1
    pfizer_arr.append(file)
  if("janssen" in tweet_text.lower() or "johnson" in tweet_text.lower()):
    num_janssen += 1
    janssen_arr.append(file)
  if("astrazeneca" in tweet_text.lower() or "vaxzevria" in tweet_text.lower()):
    num_astra += 1
    astra_arr.append(file)

  total = 112220
  other = total - num_sputnik - num_sino - num_moderna - num_pfizer - num_janssen - num_astra

print("sputnik: " + str(num_sputnik) + " percent = " + str(num_sputnik / total))
print("sino: " + str(num_sino) + " percent = " + str(num_sino / total))
print("moderna: " + str(num_moderna) + " percent = " + str(num_moderna / total))
print("pfizer: " + str(num_pfizer) + " percent = " + str(num_pfizer / total))
print("janssen: " + str(num_janssen) + " percent = " + str(num_janssen / total))
print("astrazeneca: " + str(num_astra) + " percent = " + str(num_astra / total))
print("other: " + str(other) + " percent = " + str(other / total))

sputnik: 701 percent = 0.0062466583496702905
sino: 676 percent = 0.006023881661022991
moderna: 1975 percent = 0.017599358403136697
pfizer: 5140 percent = 0.04580288718588487
janssen: 107 percent = 0.0009534842274104438
astrazeneca: 1832 percent = 0.01632507574407414
other: 101789 percent = 0.9070486544288006


In [ ]:
# Create a dataframe of based on manufacturer
moderna_df = all_tweets_df[all_tweets_df["man"] == "moderna"]
pfizer_df = all_tweets_df[all_tweets_df["man"] == "pfizer"]
astra_df = all_tweets_df[all_tweets_df["man"] == "astra"]
sinopharm_df = all_tweets_df[all_tweets_df["man"] == "sino"]
sputnik_df = all_tweets_df[all_tweets_df["man"] == "sputnik"]


In [ ]:
pfizer_df

,tweet_text,time,coords,place,tweet_id,man,sentiment,language,datetime,sent_category
6,A huge shoutout to all the heroes who particip...,2020-12-17 22:11:59+00:00,None,None,1339694860220002304,pfizer,[[0.00133509 0.00973595 0.988929 ]],en,2020-12-17,2
19,@BlogRedRobin @bolshevik_bee I was fine after ...,2021-02-18 09:18:48+00:00,None,None,1362330718068740100,pfizer,[[0.82487637 0.16976237 0.00536129]],en,2021-02-18,0
21,💾 ⓈⓀⓎ ⓉⒺⒸⒽ Science behind Pfizer vaccine can b...,2021-03-21 16:33:12+00:00,"{'type': 'Point', 'coordinates': [-4.309626, 5...","{'type': 'Point', 'coordinates': [-4.309626, 5...",1373674060220137473,pfizer,[[0.5223276 0.47188225 0.00579015]],en,2021-03-21,0
31,A family of Muslim immigrants from Turkey newl...,2020-12-06 18:10:25+00:00,None,None,1335647800428621824,pfizer,[[0.0019711 0.38696942 0.61105955]],en,2020-12-06,2
61,"Chills, fever and body aches 8 hours after get...",2021-02-08 20:58:21+00:00,None,None,1358882884707364868,pfizer,[[0.92319757 0.07217274 0.00462964]],en,2021-02-08,0
...,...,...,...,...,...,...,...,...,...,...
114430,"@CraigKellyMP Have a look at the Pfizer Study,...",2021-07-20 14:28:17+00:00,None,None,1417491558782341126,pfizer,[[0.21000953 0.78149736 0.00849314]],en,2021-07-20,1
114475,@PMOIndia @narendramodi you hv already botched...,2021-05-13 13:23:37+00:00,None,None,1392832908931391496,pfizer,[[0.8628106 0.13352172 0.00366767]],en,2021-05-13,0
114508,Pfizer's Portuguese unit said on Wednesday the...,2020-12-02 15:05:59+00:00,None,None,1334151835658149889,pfizer,[[0.00162285 0.8427091 0.15566802]],en,2020-12-02,1
114517,"2.5 hours post-2nd Pfizer vaccine, zero sympto...",2021-01-21 23:51:29+00:00,None,None,1352403473325641728,pfizer,[[0.00416999 0.5156055 0.48022446]],en,2021-01-21,1


In [ ]:
# moderna
moderna_list_pos_df = moderna_df[moderna_df["sent_category"] == 2]
moderna_list_pos = list(moderna_list_pos_df["tweet_text"])

moderna_list_neg_df = moderna_df[moderna_df["sent_category"] == 0]
moderna_list_neg = list(moderna_list_neg_df["tweet_text"])
print("Positive Moderna: ", len(moderna_list_pos), " Negative Moderna: ", len(moderna_list_neg))

# pfizer
pfizer_list_pos_df = pfizer_df[pfizer_df["sent_category"] == 2]
pfizer_list_pos = list(pfizer_list_pos_df["tweet_text"])

pfizer_list_neg_df = pfizer_df[pfizer_df["sent_category"] == 0]
pfizer_list_neg = list(pfizer_list_neg_df["tweet_text"])
print("Positive Pfizer: ", len(pfizer_list_pos), " Negative Pfizer: ", len(pfizer_list_neg))

# astrazeneca
astra_list_pos_df = astra_df[astra_df["sent_category"] == 2]
astra_list_pos = list(astra_list_pos_df["tweet_text"])

astra_list_neg_df = astra_df[astra_df["sent_category"] == 0]
astra_list_neg = list(astra_list_neg_df["tweet_text"])
print("Positive Astra: ", len(astra_list_pos), " Negative Astra: ", len(astra_list_neg))

# sputnik
sputnik_list_pos_df = sputnik_df[sputnik_df["sent_category"] == 2]
sputnik_list_pos = list(sputnik_list_pos_df["tweet_text"])

sputnik_list_neg_df = sputnik_df[sputnik_df["sent_category"] == 0]
sputnik_list_neg = list(sputnik_list_neg_df["tweet_text"])
print("Positive Sputnik: ", len(sputnik_list_pos), " Negative Sputnik: ", len(sputnik_list_neg))

# sinopharm
sinopharm_list_pos_df = sinopharm_df[sinopharm_df["sent_category"] == 2]
sinopharm_list_pos = list(sinopharm_list_pos_df["tweet_text"])

sinopharm_list_neg_df = sinopharm_df[sinopharm_df["sent_category"] == 0]
sinopharm_list_neg = list(sinopharm_list_neg_df["tweet_text"])
print("Positive Sinopharm: ", len(sinopharm_list_pos), " Negative Sinopharm: ", len(sinopharm_list_neg))

Positive Moderna:  336  Negative Moderna:  183
Positive Pfizer:  826  Negative Pfizer:  797
Positive Astra:  234  Negative Astra:  212
Positive Sputnik:  39  Negative Sputnik:  57
Positive Sinopharm:  43  Negative Sinopharm:  48


In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 
from transformers import AutoModel, AutoTokenizer


In [ ]:
covid_stopwords = ["amp","19","https","covidvaccine","modernavaccine","covid", "moderna", "pfizer", "astra", "astrazeneca", "oxfordastrazeneca","oxford","sputnik","sinovac", "sinopharm", "vaccine", "covid19", "coronavirus", "vaccine", "vaccines"]
stop_words = text.ENGLISH_STOP_WORDS.union(covid_stopwords)
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")


vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stop_words)

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer_model, embedding_model=bertweet)


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Positive Moderna
topics, probs = topic_model.fit_transform(moderna_list_pos)
topic_model.visualize_barchart(top_n_topics=5)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2022-11-08 13:31:17,509 - BERTopic - Transformed documents to Embeddings
2022-11-08 13:31:19,723 - BERTopic - Reduced dimensionality
2022-11-08 13:31:19,745 - BERTopic - Clustered reduced embeddings


In [ ]:
# Negative Moderna
topics, probs = topic_model.fit_transform(moderna_list_neg)
topic_model.visualize_barchart(top_n_topics=5)

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

2022-11-08 13:31:36,410 - BERTopic - Transformed documents to Embeddings
2022-11-08 13:31:38,361 - BERTopic - Reduced dimensionality
2022-11-08 13:31:38,378 - BERTopic - Clustered reduced embeddings


In [ ]:
# Positive Pfizer
topics, probs = topic_model.fit_transform(pfizer_list_pos)
topic_model.visualize_barchart(top_n_topics=5)

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

2022-11-08 13:32:36,788 - BERTopic - Transformed documents to Embeddings
2022-11-08 13:32:40,075 - BERTopic - Reduced dimensionality
2022-11-08 13:32:40,119 - BERTopic - Clustered reduced embeddings


In [ ]:
# Negative Pfizer
topics, probs = topic_model.fit_transform(pfizer_list_neg)
topic_model.visualize_barchart(top_n_topics=5)

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

2022-11-08 13:33:38,486 - BERTopic - Transformed documents to Embeddings
2022-11-08 13:33:42,251 - BERTopic - Reduced dimensionality
2022-11-08 13:33:42,312 - BERTopic - Clustered reduced embeddings


In [ ]:
# Positive Astra
topics, probs = topic_model.fit_transform(astra_list_pos)
topic_model.visualize_barchart(top_n_topics=5)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2022-11-08 13:34:00,504 - BERTopic - Transformed documents to Embeddings
2022-11-08 13:34:02,538 - BERTopic - Reduced dimensionality
2022-11-08 13:34:02,557 - BERTopic - Clustered reduced embeddings


In [ ]:
# Negative Astra
topics, probs = topic_model.fit_transform(astra_list_neg)
topic_model.visualize_barchart(top_n_topics=5)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

2022-11-08 13:34:18,131 - BERTopic - Transformed documents to Embeddings
2022-11-08 13:34:20,128 - BERTopic - Reduced dimensionality
2022-11-08 13:34:20,147 - BERTopic - Clustered reduced embeddings
